In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# Import the loading function of Scikit-Image
from skimage import io

## Loading the image and plotting it

By now you should be familiar with this part.

In [ ]:
img = io.imread('face.jpg')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)

## Detecting faces

This subpart is heavily based on the tutorial of dlib http://dlib.net/face_detector.py.html

In [ ]:
# Import the dlib library
import dlib
# Load the frontal face detector
detector = dlib.get_frontal_face_detector()

In [ ]:
# Apply the detector to the img and return the detections (the '1' is a upsampling factor to get better results, not mandatory)
dets = detector(img, 1)
print("Number of faces detected: {}".format(len(dets)))

`dets` is a special `dlib` structure which contains the detected rectangles

In [ ]:
# Accessing the first detected face
dets[0]

In [ ]:
from matplotlib.patches import Rectangle
def draw_detection(det):
    # gca -> get-current-axis
    current_axis = plt.gca()
    # Add a rectangle on top of the image with the position defined by the detected face
    current_axis.add_patch(
                Rectangle(
                    (det.left(), det.top()),  # x, y
                    det.right() - det.left(), det.bottom() - det.top(),  # w, h
                    edgecolor="red", fill=False))
    
plt.figure(figsize=(10,10))
plt.imshow(img)
for det in dets:
    # For each detected face, draw it
    draw_detection(det)

From what you have learned in the first session, would you be able to crop the image to get the first detected face?

Remember that the image array (in numpy) is y-first, x-second

## Extracting a face descriptor

This part of the tutorial is based on http://dlib.net/face_recognition.py.html

In [ ]:
# Shape predictor to refine the face detection result (find face landmarks like eye corners, mouth, etc...)
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
# Face recognition model
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

Loading the image and detecting the only face, you should be familiar about it by now

In [ ]:
query_img = io.imread('query.jpg')

In [ ]:
query_det = detector(query_img, 1)[0]

plt.figure(figsize=(10,10))
plt.imshow(query_img)
draw_detection(query_det)

In [ ]:
# Get the landmarks/parts for the face in box d.
query_shape = sp(query_img, query_det)
# Extract the face descriptor
query_face_descriptor = facerec.compute_face_descriptor(query_img, query_shape)
query_face_descriptor = np.asarray(query_face_descriptor)  # Converting the descriptor to standard numpy array

We extracted a 128-D vector that represents the identity of the face. But for us it is just a list of numbers that does not tell us much.

In [ ]:
print(query_face_descriptor.shape)
print(query_face_descriptor)

## Face recognition

The face descriptor is not useful by itself, but it is a great tool to compare the facial identity between different photographs.

We have 10 faces of known personalities in the `faces` directory (you can browse the directory from the jupyter system), and we will try to compare them all with our query face.

In practice, an euclidean distance between 2 descriptors smaller than 0.6 usually means that the two descriptors are from the same person.

##### Task

You will need to fill some gaps using the examples above. For each image, perform the detection, extract the face feature, and compute the euclidean distance with the face feature of the query.

Each output should look like this :

![result](desired_result.jpg)

In [ ]:
from glob import glob
from tqdm import tqdm  # For the eye-candy progress-bar

# List all the filenames in the faces directory
face_files = glob('faces/*')
face_files

In [ ]:
# For each file
for filename in tqdm(face_files, 'Processing'):
    # Read the image
    img = io.imread(filename)
    # Detect faces in img, and take the first detected result
    det = None  # TODO
    # Refine the detected face by extracting the 
    shape = None  # TODO
    # Extract the face descriptor
    face_descriptor = None  # TODO
    # Compute the euclidean distance between our query face descriptor and the current one
    dist = None  # TODO
    # Plot the detection result
    plt.figure()
    plt.imshow(img)
    draw_detection(det)
    plt.title("{} : distance={:.2f}".format(filename.split('/')[-1], dist))

If all goes well, despite the original image and the one in the 'face database' being quite different, the dlib library is quite confident about both of them coming from the same person (d<0.6).

Funny thing, the second closest face descriptor comes from his father...

In [ ]:
# For the bored people, you could have a better look at the shape predictor that finds out the landmark on the face
# A good starting point is http://dlib.net/face_landmark_detection.py.html